In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

import random
random.seed(7)


def generator(X, Y, batch_size=32, train=True):
    while True:
        for offset in range(0, len(X), batch_size):
            X_batch = np.stack(X[offset:offset+batch_size], axis=0)
            Y_batch = np.stack(Y[offset:offset+batch_size], axis=0)

            Y_batch_ = np.empty((Y_batch.shape[0], 2,2,6))
            for m in range(Y_batch.shape[0]):
                for i in range(6):
                    Y_batch_[m, :,:,i] = [[np.sum(Y_batch[m, :3, :3, i]), np.sum(Y_batch[m, :3, 3:, i])], [np.sum(Y_batch[m, 3:, :3, i]), np.sum(Y_batch[m, 3:, 3:, i])]]
            Y_batch_[Y_batch_ > 0] = 1

            if train:
                Y_f = np.array([Y_batch_[e].flatten() for e in range(Y_batch_.shape[0])])
                yield (X_batch, Y_f)
            else:
                yield X_batch

structure_ids = []
for line in open('./structures lists/structures human.txt', 'r'):
    line = line.strip('\n')
    structure_ids.append(line)
# for line in open('./structures lists/structures ecoli.txt', 'r'):
#     line = line.strip('\n')
#     structure_ids.append(line)
structure_ids.remove('4pkd')
structure_ids.remove('1a9n')
structure_ids.remove('2adc')
random.shuffle(structure_ids)
print(len(structure_ids))

X_train = []
X_test_pos = []
X_test_neg = []
Y_train = []
Y_test_pos = []
Y_test_neg = []
num_aa_train = 0
num_aa_test_pos = 0
num_aa_test_neg = 0
num_train = int(len(structure_ids)*0.7)
for i, structure_id in enumerate(structure_ids):
    protein = np.load('../data/voxelized data 6x6x6/' + structure_id + '_protein.npy', mmap_mode='r')
    rna = np.load('../data/voxelized data 6x6x6/' + structure_id + '_rna_3D.npy', mmap_mode='r')
   
    na = 0
    pos = 0
    while (np.sum(rna[na]) > 100) and (na < len(rna)-1):
        pos +=1
        na +=1

    if i <= num_train:
        X_train.extend(protein[:pos, :, :, :, :3])
        Y_train.extend(rna[:pos])
        num_aa_train +=pos
#         if pos > len(rna)/2:
#             X_train.extend(protein[:, :, :, :, :3])
#             Y_train.extend(rna)
#             num_aa_train +=len(rna)
#         else:
#             X_train.extend(protein[:pos, :, :, :, :3])
#             X_train.extend(protein[-pos:, :, :, :, :3])
#             Y_train.extend(rna[:pos])
#             Y_train.extend(rna[-pos:])
#             num_aa_train +=2*pos
    else:
        X_test_pos.extend(protein[:pos, :, :, :, :3])
        Y_test_pos.extend(rna[:pos])
        X_test_neg.extend(protein[pos:, :, :, :, :3])
        Y_test_neg.extend(rna[pos:])
        num_aa_test_pos +=pos
        num_aa_test_neg +=(len(rna)-pos)

#         if pos > len(rna)/2:
#             X_test.extend(protein[:, :, :, :, :3])
#             Y_test.extend(rna)
#             num_aa_test +=len(rna)
#         else:
#             X_test.extend(protein[:pos, :, :, :, :3])
#             X_test.extend(protein[-pos:, :, :, :, :3])
#             Y_test.extend(rna[:pos])
#             Y_test.extend(rna[-pos:])
#             num_aa_test +=2*pos


   
n_steps_train = int(num_aa_train/400) 

generator_train = generator(X_train, Y_train, 400, True)

ins = tf.keras.layers.Input((6, 6, 6, 3))
con1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(ins)
con2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con1)
con3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con2)
maxp1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con3)
con4 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp1)
con5 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con4)
con6 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con5)
maxp2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con6)
con7 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp2)
con8 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu')(con7)
con9 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu')(con8)
# maxp3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con9)
batch = tf.keras.layers.BatchNormalization()(con9)
flat = tf.keras.layers.Flatten()(batch)
dens2 = tf.keras.layers.Dense(units=256, activation='relu')(flat)
drop2 = tf.keras.layers.Dropout(0.6)(dens2)
outs = tf.keras.layers.Dense(units=24, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=ins, outputs=outs)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])

model.summary()

# checkpoint
# filepath="weights_best.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# model.fit(X_train, Y_train_f, validation_split=0.33, epochs=1, batch_size=200, callbacks=callbacks_list, verbose=0)
model.fit_generator(generator_train, steps_per_epoch=n_steps_train, epochs=500, callbacks=None, verbose=1, max_queue_size=2)


def predict(X_test, Y_test, num_aa_test):
    Y_test = np.stack(Y_test, axis=0)
    Y_test_ = np.empty((len(Y_test), 2,2,10))
    for m in range(len(Y_test)):
        for i in range(6):
            Y_test_[m, :,:,i] = [[np.sum(Y_test[m, :3, :3, i]), np.sum(Y_test[m, :3, 3:, i])], [np.sum(Y_test[m, 3:, :3, i]), np.sum(Y_test[m, 3:, 3:, i])]]
    Y_test_[Y_test_ > 0] = 1

    n_steps_test = int(num_aa_test/400)
    generator_test = generator(X_test, Y_test, 400, False)

    # model_best = model
    # model_best.load_weights("weights_best.hdf5")
    # print(model.evaluate(X_test, Y_test, verbose=0, batch_size=100))
    # model_best.save('model_cnn_15_2.h5')
    # Y_pred = model_best.predict(X_test, batch_size=200)
    Y_pred = model.predict_generator(generator_test, steps=n_steps_test)
    print(Y_pred.shape)
    Y_pred_ = np.array([Y_pred[i].reshape((2,2,6)) for i in range(Y_pred.shape[0])])

    #CNN
    Y_pred_prob = Y_pred_
    Y_pred_[Y_pred_ >= 0.5] = 1
    Y_pred_[Y_pred_ < 0.5] = 0

    print(Y_pred_.shape)
    print(Y_test_.shape)
    Y_test_ = Y_test_[:Y_pred_.shape[0]]

    print('CNN: \n')
    for i in range(6):
        confusion_matrix = [sklearn.metrics.confusion_matrix(Y_test_[:,l , c, i], Y_pred_[:,l , c, i]) for l in range(2) for c in range(2)]
        accuracy = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix]
        auc = [sklearn.metrics.roc_auc_score(Y_test_[:,l , c, i], Y_pred_prob[:,l , c, i]) for l in range(2) for c in range(2)]

        print(f'level {i}')
        for q in range(len(confusion_matrix)):
            print(confusion_matrix[q], np.round(accuracy[q], 2), np.round(auc[q], 2))

    # baseline model
    # predict all zeros; at least 50% correct predictions because there are 1/2 of negative examples
    Y_pred_base = np.zeros(Y_test_.shape)

    # po = np.sum(Y_train, axis=0)/Y_train.shape[0]
    # po[po >= 0.5] = 1
    # po[po < 0.5] = 0
    # Y_pred_base = np.tile(po, (Y_test.shape[0],1))

    Y_pred_base[Y_pred_base >= 0.5] = 1
    Y_pred_base[Y_pred_base < 0.5] = 0
    print(Y_pred_base.shape)
    print(f'\n BASELINE MODEL: \n')
    for i in range(6):
        confusion_matrix_base = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]
        accuracy_base = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_base]
        auc_base = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]

        print(f'level {i}')
        for q in range(len(confusion_matrix_base)):
            print(confusion_matrix_base[q], np.round(accuracy_base[q], 2), np.round(auc_base[q], 2))

    #random model
    Y_pred_random = np.random.random(Y_test_.shape)
    Y_pred_random[Y_pred_random >= 0.5] = 1
    Y_pred_random[Y_pred_random < 0.5] = 0

    print(f'\n RANDOM MODEL: \n')
    for i in range(6):
        confusion_matrix_random = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]
        accuracy_random = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_random]
        auc_random = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]

        print(f'level {i}')
        for q in range(len(confusion_matrix_random)):
            print(confusion_matrix_random[q], np.round(accuracy_random[q], 2), np.round(auc_random[q], 2))
        
predict(X_test_pos, Y_test_pos, num_aa_test_pos)
print('\n')
predict(X_test_neg, Y_test_neg, num_aa_test_neg)

230
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 6, 6, 6, 3)        0         
_________________________________________________________________
conv3d_90 (Conv3D)           (None, 6, 6, 6, 64)       5248      
_________________________________________________________________
conv3d_91 (Conv3D)           (None, 6, 6, 6, 32)       55328     
_________________________________________________________________
conv3d_92 (Conv3D)           (None, 6, 6, 6, 32)       27680     
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 3, 3, 3, 32)       0         
_________________________________________________________________
conv3d_93 (Conv3D)           (None, 3, 3, 3, 32)       27680     
_________________________________________________________________
conv3d_94 (Conv3D)           (None, 3, 3, 3, 16)       13840     
______

55/55 [==============================] - 4s 67ms/step - loss: 0.6417 - acc: 0.6511
Epoch 59/500
55/55 [==============================] - 4s 68ms/step - loss: 0.6398 - acc: 0.6547
Epoch 60/500
55/55 [==============================] - 4s 77ms/step - loss: 0.6391 - acc: 0.6557
Epoch 61/500
55/55 [==============================] - 4s 78ms/step - loss: 0.6396 - acc: 0.6515
Epoch 62/500
55/55 [==============================] - 4s 72ms/step - loss: 0.6393 - acc: 0.6533
Epoch 63/500
55/55 [==============================] - 4s 68ms/step - loss: 0.6379 - acc: 0.6551
Epoch 64/500
55/55 [==============================] - 4s 68ms/step - loss: 0.6365 - acc: 0.6572
Epoch 65/500
55/55 [==============================] - 4s 65ms/step - loss: 0.6361 - acc: 0.6565
Epoch 66/500
55/55 [==============================] - 4s 67ms/step - loss: 0.6353 - acc: 0.6577
Epoch 67/500
55/55 [==============================] - 4s 71ms/step - loss: 0.6348 - acc: 0.6577: 0s - loss: 0.6365 -
Epoch 68/500
55/55 [============

55/55 [==============================] - 4s 74ms/step - loss: 0.5859 - acc: 0.6919
Epoch 142/500
55/55 [==============================] - 4s 73ms/step - loss: 0.5860 - acc: 0.6919
Epoch 143/500
55/55 [==============================] - 4s 72ms/step - loss: 0.5845 - acc: 0.6929
Epoch 144/500
55/55 [==============================] - 4s 67ms/step - loss: 0.5835 - acc: 0.6932: 0s - loss: 0.5831 - acc: 0.69
Epoch 145/500
55/55 [==============================] - 4s 70ms/step - loss: 0.5836 - acc: 0.6937
Epoch 146/500
55/55 [==============================] - 4s 69ms/step - loss: 0.5825 - acc: 0.6944
Epoch 147/500
55/55 [==============================] - 4s 72ms/step - loss: 0.5808 - acc: 0.6963
Epoch 148/500
55/55 [==============================] - 4s 78ms/step - loss: 0.5817 - acc: 0.6947
Epoch 149/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5814 - acc: 0.6943: 2s 
Epoch 150/500
55/55 [==============================] - 4s 69ms/step - loss: 0.5818 - acc: 0.6931
Epoch 15

55/55 [==============================] - 4s 74ms/step - loss: 0.5540 - acc: 0.7089
Epoch 222/500
55/55 [==============================] - 4s 67ms/step - loss: 0.5505 - acc: 0.7125
Epoch 223/500
55/55 [==============================] - 4s 80ms/step - loss: 0.5522 - acc: 0.7102
Epoch 224/500
55/55 [==============================] - 4s 74ms/step - loss: 0.5513 - acc: 0.7108
Epoch 225/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5522 - acc: 0.7101
Epoch 226/500
55/55 [==============================] - 4s 69ms/step - loss: 0.5519 - acc: 0.7104
Epoch 227/500
55/55 [==============================] - 4s 68ms/step - loss: 0.5496 - acc: 0.7132
Epoch 228/500
55/55 [==============================] - 4s 73ms/step - loss: 0.5485 - acc: 0.7144
Epoch 229/500
55/55 [==============================] - 4s 72ms/step - loss: 0.5516 - acc: 0.7104
Epoch 230/500
55/55 [==============================] - 4s 71ms/step - loss: 0.5521 - acc: 0.7096
Epoch 231/500
55/55 [=======================

55/55 [==============================] - 4s 67ms/step - loss: 0.5361 - acc: 0.7190
Epoch 305/500
55/55 [==============================] - 4s 71ms/step - loss: 0.5365 - acc: 0.7192: 1s - lo
Epoch 306/500
55/55 [==============================] - 4s 71ms/step - loss: 0.5352 - acc: 0.7203
Epoch 307/500
55/55 [==============================] - 4s 73ms/step - loss: 0.5350 - acc: 0.7202
Epoch 308/500
55/55 [==============================] - 4s 67ms/step - loss: 0.5343 - acc: 0.7210
Epoch 309/500
55/55 [==============================] - 4s 69ms/step - loss: 0.5347 - acc: 0.7205
Epoch 310/500
55/55 [==============================] - 4s 70ms/step - loss: 0.5350 - acc: 0.7200
Epoch 311/500
55/55 [==============================] - 4s 67ms/step - loss: 0.5335 - acc: 0.7208
Epoch 312/500
55/55 [==============================] - 4s 64ms/step - loss: 0.5334 - acc: 0.7210
Epoch 313/500
55/55 [==============================] - 4s 68ms/step - loss: 0.5333 - acc: 0.7216: 0s - loss: 0.5379 - 
Epoch 314/500

55/55 [==============================] - 4s 68ms/step - loss: 0.5182 - acc: 0.7311
Epoch 388/500
55/55 [==============================] - 4s 72ms/step - loss: 0.5218 - acc: 0.7272
Epoch 389/500
55/55 [==============================] - 4s 68ms/step - loss: 0.5205 - acc: 0.7287
Epoch 390/500
55/55 [==============================] - 4s 68ms/step - loss: 0.5176 - acc: 0.7309
Epoch 391/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5201 - acc: 0.7293
Epoch 392/500
55/55 [==============================] - 4s 74ms/step - loss: 0.5187 - acc: 0.7296
Epoch 393/500
55/55 [==============================] - 4s 67ms/step - loss: 0.5202 - acc: 0.7284
Epoch 394/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5195 - acc: 0.7292
Epoch 395/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5177 - acc: 0.7311
Epoch 396/500
55/55 [==============================] - 4s 68ms/step - loss: 0.5164 - acc: 0.7330
Epoch 397/500
55/55 [=======================

55/55 [==============================] - 4s 67ms/step - loss: 0.5107 - acc: 0.7333
Epoch 469/500
55/55 [==============================] - 4s 69ms/step - loss: 0.5079 - acc: 0.7351: 1s - los
Epoch 470/500
55/55 [==============================] - 4s 67ms/step - loss: 0.5113 - acc: 0.7332: 0s - loss: 0.5115 - acc: 0.732
Epoch 471/500
55/55 [==============================] - 4s 74ms/step - loss: 0.5106 - acc: 0.7331
Epoch 472/500
55/55 [==============================] - 4s 72ms/step - loss: 0.5118 - acc: 0.7322
Epoch 473/500
55/55 [==============================] - 4s 70ms/step - loss: 0.5123 - acc: 0.7323: 2
Epoch 474/500
55/55 [==============================] - 4s 73ms/step - loss: 0.5111 - acc: 0.7329
Epoch 475/500
55/55 [==============================] - 4s 69ms/step - loss: 0.5112 - acc: 0.7330
Epoch 476/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5104 - acc: 0.7336
Epoch 477/500
55/55 [==============================] - 4s 66ms/step - loss: 0.5102 - acc: 0.7336

/home/skuznik/local/miniconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:138: RuntimeWarning: invalid value encountered in greater


(12800, 24)
(12800, 2, 2, 6)
(13056, 2, 2, 10)
CNN: 

level 0
[[9539    0]
 [3261    0]] 0.75 0.5
[[9343    0]
 [3457    0]] 0.73 0.5
[[9651    0]
 [3149    0]] 0.75 0.5
[[9492    0]
 [3308    0]] 0.74 0.5
level 1
[[6957 1894]
 [2478 1471]] 0.66 0.58
[[8718    0]
 [4082    0]] 0.68 0.5
[[8909  165]
 [3553  173]] 0.71 0.51
[[8699  304]
 [3373  424]] 0.71 0.54
level 2
[[4352 3890]
 [1649 2909]] 0.57 0.58
[[3767 4329]
 [1286 3418]] 0.56 0.6
[[7956  450]
 [3980  414]] 0.65 0.52
[[7607  583]
 [3770  840]] 0.66 0.56
level 3
[[2688 4529]
 [1221 4362]] 0.55 0.58
[[3046 4224]
 [ 969 4561]] 0.59 0.62
[[6860  626]
 [4610  704]] 0.59 0.52
[[6005 1236]
 [3658 1901]] 0.62 0.59
level 4
[[1977 4734]
 [ 626 5463]] 0.58 0.6
[[3039 3574]
 [1107 5080]] 0.63 0.64
[[5881 1168]
 [4262 1489]] 0.58 0.55
[[5300 1344]
 [3436 2720]] 0.63 0.62
level 5
[[1890 4871]
 [ 643 5396]] 0.57 0.59
[[3015 3595]
 [1228 4962]] 0.62 0.63
[[5500 1634]
 [3955 1711]] 0.56 0.54
[[5239 1665]
 [3070 2826]] 0.63 0.62
(12800, 2, 2, 10)